In [1]:
import pandas as pd
from googleapiclient.discovery import build
import googleapiclient.discovery
import googleapiclient.errors
import requests

In [2]:
# Set up the API key and YouTube API

API_KEY = open('youtube_api_3.txt', 'r').read()
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [3]:
def check_comments_enabled(video_id):
    video_url = f'https://www.googleapis.com/youtube/v3/videos?part=status&id={video_id}&key={API_KEY}'
    video_response = requests.get(video_url)
    video_data = video_response.json()
    print("HI")
    print(video_data)
    if 'items' in video_data and len(video_data['items']) > 0:
        return not video_data['items'][0]['status'].get('commentsDisabled', False)
    else:
        print(f'Video not found for ID: {video_id}')
        return False  # Default to False if video not found

In [4]:
def search_videos(query, max_results=50):
    """Search for videos on a specific topic."""
    search_response = youtube.search().list(
        q=query,
        type='video',
        part='id',
        maxResults=max_results
    ).execute()
    video_ids = [item['id']['videoId'] for item in search_response['items']]
    return video_ids

In [5]:
def get_comments(video_id):
    """Get comments for a given video ID."""
    comments = []

    try: # Wrap the API call in a try-except block
        # Retrieve the comments
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            textFormat='plainText',
            maxResults =50,
            order = 'relevance' #,
            #searchTerms = 'Team Fortress 2'
        ).execute()

        if response:
            for item in response['items']:
                text = item['snippet']['topLevelComment']['snippet']['textDisplay']
                channel_id = item['snippet']['topLevelComment']['snippet']['channelId']
                like_count = item['snippet']['topLevelComment']['snippet']['likeCount']
                channel_id_text = item['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
                comment = {
                    'video_id': video_id,
                    'text': text,
                    'channel_id': channel_id,
                    'like_count': like_count,
                    'channel_id_text': channel_id_text
                }
                comments.append(comment)

    except googleapiclient.errors.HttpError as e:
        # Handle the HttpError, specifically when comments are disabled
        if e.resp.status == 403 and "commentsDisabled" in str(e):
            print(f"Comments are disabled for video ID: {video_id}")
        elif e.resp.status == 404: # Check for video not found error
            print(f"Video not found for video ID: {video_id}")
        else:
            # Re-raise the exception for other errors
            raise e
    return comments

In [6]:
def get_video_info(video_id):
  """Get video information for a given video ID."""
  # Step 1: Get video details
  video_url = f'https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&id={video_id}&key={API_KEY}'
  video_response = requests.get(video_url)
  video_data = video_response.json()

  if 'items' in video_data and len(video_data['items']) > 0:
      channel_id = video_data['items'][0]['snippet']['channelId']
      channel_name = video_data['items'][0]['snippet']['channelTitle']
      views = video_data['items'][0]['statistics'].get('viewCount', 'N/A')
      likes = video_data['items'][0]['statistics'].get('likeCount', 'N/A')

      # Step 2: Get channel details
      channel_url = f'https://www.googleapis.com/youtube/v3/channels?part=statistics&id={channel_id}&key={API_KEY}'
      channel_response = requests.get(channel_url)
      channel_data = channel_response.json()

      if 'items' in channel_data and len(channel_data['items']) > 0:
          subscribers = channel_data['items'][0]['statistics'].get('subscriberCount', 'N/A')
      else:
          subscribers = 'N/A'
  else:
      print('Video not found or no data available.')
      channel_name, views, likes, subscribers = 'N/A', 'N/A', 'N/A', 'N/A'


  # Step 3: Create a list
  data = {
      'Channel Name': channel_name,
      'Views': int(views) if views != 'N/A' else -1,
      'Likes': int(likes)if likes != 'N/A' else -1,
      'Subscribers': int(subscribers) if subscribers != 'N/A' else -1
  }
  return data

In [7]:
def perform_data_collection(topic,max_results):
  # Get the top  video IDs
  print(f"Searching for videos about: {topic}")
  video_ids = search_videos(topic, max_results=max_results)

  all_comments = []
  video_info_list = []

  # Iterate through the video IDs to get comments
  for video_id in video_ids:
    # Get video information
    video_info_list.append(get_video_info(video_id))
    print(f"Fetching comments for video ID: {video_id}")
    comments = get_comments(video_id)
    all_comments.extend(comments)  # Add the comments to the list
    print("length of all comments ", len(all_comments))
  return all_comments, video_info_list

In [8]:
video_list_full = []
comment_list_full = []

In [9]:
df = pd.read_csv('GamesList.csv')

# Store the titles in a variable called 'topic'
topics = df['title'].tolist()  # Convert the 'title' column to a list

# Iterate through the titles
for topic in topics:
    print(topic)
    max_results = 200
    all_comments, video_info_list = perform_data_collection(topic,max_results)
    video_list_full.extend(video_info_list)
    comment_list_full.extend(all_comments)

Snakeybus
Searching for videos about: Snakeybus
Fetching comments for video ID: xzDyhUdsuCc
length of all comments  50
Fetching comments for video ID: UNhGynJEp-M
length of all comments  100
Fetching comments for video ID: xwj8mD5MnMg
length of all comments  150
Fetching comments for video ID: zZSbH9bUVaM
length of all comments  200
Fetching comments for video ID: AsV7_nJHd-A
length of all comments  250
Fetching comments for video ID: Vb1gZyckHX8
length of all comments  300
Fetching comments for video ID: 3a2Ru1d5YCc
length of all comments  350


Fetching comments for video ID: BsjS8bmMFNU
Comments are disabled for video ID: BsjS8bmMFNU
length of all comments  350
Fetching comments for video ID: RaWn4YXZpP8
length of all comments  400
Fetching comments for video ID: A3_Xhge3qlE
length of all comments  450
Fetching comments for video ID: XxBtAU4p6GA
length of all comments  500
Fetching comments for video ID: dedWvlqgeq4
length of all comments  550
Fetching comments for video ID: hccR1OjsoAc
length of all comments  600
Fetching comments for video ID: E4vzdt4wTXc
length of all comments  650
Fetching comments for video ID: RtC0eQJoSFM
length of all comments  700
Fetching comments for video ID: AuEAoD8pYRk
length of all comments  750
Fetching comments for video ID: FJ5AG1zv0pY
length of all comments  761
Fetching comments for video ID: oNx9K26ylU4
length of all comments  811
Fetching comments for video ID: SbRFsI32Mxo
length of all comments  861
Fetching comments for video ID: Whyndp_iyZM
length of all comments  872
Fetching comment

Fetching comments for video ID: smlef-jkmuQ
Comments are disabled for video ID: smlef-jkmuQ
length of all comments  0
Fetching comments for video ID: xUko31SmXxA
length of all comments  50
Fetching comments for video ID: Ta9ReKaZOGw
length of all comments  100
Fetching comments for video ID: YSlrAuFAnhc
length of all comments  111
Fetching comments for video ID: uXhfnGv_fEQ
length of all comments  116


Fetching comments for video ID: wrE8wHZQ1BI
length of all comments  130
Fetching comments for video ID: wgR1tNgLV68
Comments are disabled for video ID: wgR1tNgLV68
length of all comments  130
Fetching comments for video ID: TjApDcGW9Io
length of all comments  180
Fetching comments for video ID: 6t-jJkMlntY
length of all comments  180
Fetching comments for video ID: hV_nResY_nE
length of all comments  230
Fetching comments for video ID: GTfYP-Y7bpM
length of all comments  280
Fetching comments for video ID: UpNid_rWDnI
length of all comments  330
Fetching comments for video ID: 9Lypkwll6tM
length of all comments  357
Fetching comments for video ID: QPLsO2KLRMc
length of all comments  361
Fetching comments for video ID: XRT--nWS9NY
length of all comments  411
Fetching comments for video ID: sMSK-AoOsGg
length of all comments  428
Fetching comments for video ID: _H3-AXp8EhI
length of all comments  431
Fetching comments for video ID: sddYEcZIiZE
length of all comments  481
Fetching comment

length of all comments  962
Fetching comments for video ID: 7_LW7u-nk6Q
Comments are disabled for video ID: 7_LW7u-nk6Q
length of all comments  962
Fetching comments for video ID: dEFG7UhYEWI
length of all comments  1012
Fetching comments for video ID: CxociTjzR4Q
length of all comments  1048
Fetching comments for video ID: EMOuxmtljIw
length of all comments  1049
Fetching comments for video ID: JLy1DYsqOUI
length of all comments  1073
Fetching comments for video ID: HSnfcBp31qc
length of all comments  1123
Fetching comments for video ID: kre5EfvpO54
length of all comments  1124
Fetching comments for video ID: 4Wlm4MGTUR0
length of all comments  1174
Fetching comments for video ID: 02ULapg4hKc
length of all comments  1224
Fetching comments for video ID: CRpPFgvbEvE
length of all comments  1224
Fetching comments for video ID: HYFCGPPjJnk
length of all comments  1274
Fetching comments for video ID: QvfK6f_8iGE
length of all comments  1324
Fetching comments for video ID: d5Xh1vK3PLo
lengt

...

Fetching comments for video ID: 0JG5Y7ZWvWU
Comments are disabled for video ID: 0JG5Y7ZWvWU
length of all comments  184
Fetching comments for video ID: YPlxlu4y0oA


length of all comments  198
Fetching comments for video ID: NMR7JovJlRo
Comments are disabled for video ID: NMR7JovJlRo
length of all comments  198
Fetching comments for video ID: InTwZnuKB0w
length of all comments  248
Fetching comments for video ID: FE8ogL4sKLQ
length of all comments  248
Fetching comments for video ID: AJLgkqTKsEE
length of all comments  265
Fetching comments for video ID: ILR2wEY_e_Q
length of all comments  315
Fetching comments for video ID: mMNnvuEzBt8
length of all comments  365
Fetching comments for video ID: CSlDgyxeNi8
length of all comments  415
Fetching comments for video ID: fABPqSAJptc
length of all comments  465
Fetching comments for video ID: 5Evd71fWQkU
length of all comments  511
Fetching comments for video ID: rSXGccP_kwM
length of all comments  561
Fetching comments for video ID: ScejdNGTXCU
length of all comments  611
Fetching comments for video ID: DfWbZACatF0
length of all comments  661
Fetching comments for video ID: EUPpdfBuGfM
length of all co

Fetching comments for video ID: nYtQxseF5tE
Comments are disabled for video ID: nYtQxseF5tE
length of all comments  1149
Fetching comments for video ID: 8ULQrQNJGhw
length of all comments  1199
Fetching comments for video ID: vEyUu-u2aFA
length of all comments  1249
Fetching comments for video ID: 4xw45qIkcv8
length of all comments  1299
Fetching comments for video ID: Ls9Sfn5R5LI
length of all comments  1349
Fetching comments for video ID: cKqNLkAyUb4
length of all comments  1368
Fetching comments for video ID: 7nv4x9pVIAY
length of all comments  1418
Fetching comments for video ID: bFt22YrfIdU
length of all comments  1468
Fetching comments for video ID: LW7rkSCoe04
length of all comments  1518
Fetching comments for video ID: rpNTSBQj84Q
length of all comments  1568
Fetching comments for video ID: _PABigAvj4c
length of all comments  1579
Fetching comments for video ID: hxPET50Sawc
length of all comments  1629
Fetching comments for video ID: oyeU8c-Embk
length of all comments  1679
Fet

length of all comments  1715
Fetching comments for video ID: QhhTdi-xmn0
Comments are disabled for video ID: QhhTdi-xmn0
length of all comments  1715
Fetching comments for video ID: von0Ux2c2ac
length of all comments  1765
Fetching comments for video ID: Xpel-V3yWzs
length of all comments  1774
Fetching comments for video ID: HDQyw9RlqG0
length of all comments  1824
Fetching comments for video ID: d__QPe16Wf8
length of all comments  1874
Fetching comments for video ID: 5tOXsBJXWOY
length of all comments  1895
Fetching comments for video ID: 92vkmjd_oYU
length of all comments  1945
Project Winter
Searching for videos about: Project Winter
Fetching comments for video ID: JBB4a4iKr8s
length of all comments  50
Fetching comments for video ID: alBMgxzYExc
length of all comments  100
Fetching comments for video ID: kcX_clkApRo
length of all comments  123
Fetching comments for video ID: 13UqMaLdD9g
length of all comments  173
Fetching comments for video ID: nJJ3k9snR7A
length of all comments 

length of all comments  230
Fetching comments for video ID: hlPoZsjLFyc
Comments are disabled for video ID: hlPoZsjLFyc
length of all comments  230
Fetching comments for video ID: eoeacno-5aw
length of all comments  243
Fetching comments for video ID: eGlhDgV-APw
length of all comments  243
Fetching comments for video ID: vMTIvxJ7bFs
length of all comments  250
Fetching comments for video ID: ayk_00tFqMQ
length of all comments  272
Fetching comments for video ID: KUdxShTNWTc
length of all comments  279
Fetching comments for video ID: LAO6JZW0a94
length of all comments  281
Fetching comments for video ID: -uhEKIQsJRE
length of all comments  286
Fetching comments for video ID: dMvzLrPq1oo
length of all comments  287
Fetching comments for video ID: gLD_wmbhago
length of all comments  291
Fetching comments for video ID: --UXA1DoZ7E
length of all comments  341
Fetching comments for video ID: 7lcYClbSdMk
length of all comments  391
Fetching comments for video ID: PCIUxH-8qAo
length of all co

In [11]:
# Convert all_comments to a df with columns as keys in the dictionary and values as each row
comments_df = pd.DataFrame(comment_list_full)
# Create a dataframe for the video information
video_info_df = pd.DataFrame(video_list_full)

In [12]:
comments_df.to_csv('youtube_comments.csv')
comments_df.shape

(68995, 5)

In [13]:
result_df = video_info_df.groupby('Channel Name', as_index=False)[['Views', 'Likes']].sum().sort_values(by=['Views', 'Likes'], ascending=False)
video_info_df.to_csv('youtube_videos.csv')